# Time Series & Historical Query Analysis

Welcome to this example where we'll demonstrate how to work with large datasets in kdb+ to analyze time-series data. 

One of the key features of kdb+ is its ability to handle huge volumes of data with exceptional speed and efficiency. Whether it's reading massive datasets, performing time-based aggregations, or joining data from different sources, kdb+ excels at time-series analysis. By the end of this example, you'll have a clear understanding of how to create, manipulate, store, and analyze data using kdb+/q. Along the way, we'll introduce several key concepts that are fundamental to working with kdb+/q.


Here, we'll cover:
- Creating a large time-series dataset from scratch
- Saving this data to a database on disk
- Scaling database to 1 Billion rows
- Performing time-based aggregations to analyze trends over time
- Using asof joins (aj) to combine time-series data (e.g., matching trades to quotes)

## 1. Prerequisites

1. For setup instructions and prerequisites, please refer to the [README](README.md).
2. Ensure PyKX is properly initialized by running the cell below.<br/>
   <b>Note</b>: This is a Python cell that will enable the kernel to execute q code as the default language for all later cells.

In [1]:
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


## 2. Create the Time Series Dataset

Let’s start by creating a sample dataset to work with. This dataset will simulate trade data over a period of time, with random values for price, size, and symbols. We’ll generate 20 million rows of trade data.

In [2]:
n:20000000
syms:100?`3
day:2025.01.01
trade:([] 
    time:asc (`timestamp$day) + n?24:00:00;              // Start from midnight, spread across 24h
    sym:n?syms;                                          // Random stock tickers
    price:n?100f;                                        // Random trade prices
    size:n?1000                                          // Random trade sizes
 )

Here's a breakdown of what's happening:
- `n: 2000000` sets the number of rows we want to generate
- We define a new table with table notation `([] col1:<values>; col2:<values>: ...)`
- We use `?` to generate random values for 4 columns:
    - `time` is populated with timestamps starting from midnight and increasing across a 24-hour period, with a random offset to simulate a spread of trades.
    - `sym` is populated with random symbols, selected from a list.
    - `price` and trade `size` are randomnly generated

This table is now available in memory to investigate and query. Let's take a quick look at the row [`count`](#https://code.kx.com/q/ref/count/), schema details with [`meta`](#https://code.kx.com/q/ref/meta/) and first 10 rows using [`sublist`](#https://code.kx.com/q/ref/sublist/).

These simple commands are essential when exploring your data quickly in kdb+/q.

In [3]:
count trade              // get row count

20000000


In [4]:
meta trade               // get table schema details - datatypes, column names etc

c    | t f a
-----| -----
time | p   s
sym  | s    
price| f    
size | j    


The following columns are produced when we run `meta`:
- c: column name
- t: <a href="https://code.kx.com/q/ref/#datatypes" target="_blank">column type</a>
- f: <a href="https://code.kx.com/q4m3/8_Tables/#85-foreign-keys-and-virtual-columns" target="_blank">foreign keys</a>
- a: <a href="https://code.kx.com/q/ref/set-attribute/" target="_blank">attributes</a> (modifiers applied for performance optimisation)

In [5]:
10 sublist trade         // get first 10 rows 

time                          sym price    size
-----------------------------------------------
2025.01.01D00:00:00.000000000 fln 1.967696 82  
2025.01.01D00:00:00.000000000 jpa 59.20225 505 
2025.01.01D00:00:00.000000000 lag 4.720372 285 
2025.01.01D00:00:00.000000000 lig 10.19818 706 
2025.01.01D00:00:00.000000000 had 92.14602 267 
2025.01.01D00:00:00.000000000 jfo 5.953175 490 
2025.01.01D00:00:00.000000000 hmd 8.237707 167 
2025.01.01D00:00:00.000000000 pek 5.712376 457 
2025.01.01D00:00:00.000000000 bgj 48.08972 535 
2025.01.01D00:00:00.000000000 fna 27.63048 248 


## 3.  Save Data to Disk

Once the data is generated, you’ll likely want to save it to disk for persistent storage.

Because we want the ability to scale, partitioning by date will be a good approach for this dataset. Without partitioning, queries that span large time periods would require scanning entire datasets, which can be very slow and resource-intensive. By partitioning data, kdb+ can limit the query scope to the relevant partitions, significantly speeding up the process.

To partition by date we can use the inbuilt function [`.Q.dpft`](#https://code.kx.com/q/ref/dotq/#dpft-save-table) to save the data to disk - this may take ~20 seconds to complete.


In [6]:
homeDir:getenv[`HOME]                   // Get the home directory for edu.kx.com
dbDir:homeDir,"/data"                   // Define database location as string
dbPath:hsym `$dbDir                     // Database location as hsym for file I/O

In [7]:
.z.zd:(17;2;6)                          // Set compression level

In [ ]:
.Q.dpft[dbPath;day;`sym;`trade]         // Save data as a partitioned database

In the above:
- <a href="https://code.kx.com/q/ref/hsym/" target="_blank">hsym</a>: This function prefixes the directory location with a colon to make it a file handle
- <a href="https://code.kx.com/q/ref/dotz/#zzd-compressionencryption-defaults" target="_blank">.z.d</a>: This function sets the compression parameters.
- <a href="https://code.kx.com/q/ref/dotq/#dpft-save-table" target="_blank">.Q.dpft[d;p;f;t]</a>: This command saves data to a <b>(d)</b>atabase location, targeting a particular <b>(p)</b>artition and indexes the data on a chosen <b>(f)</b>ield for the specified <b>(t)</b>able.

One persisted, the table name is returned. We can test its worked as expected by deleting the `trade` table we have in memory and reloading the database from disk.

In [9]:
delete trade from `.                     // Delete in memory table
system"l ",dbDir                         // Load the partitioned database
meta trade                               // Check it exists

.
c    | t f a
-----| -----
date | d    
sym  | s   p
time | p    
price| f    
size | j    


kdb+ actually offers a number of different methods to store tables which will allow for efficient storage and querying for different sized datasets: flat, splayed, partitioned and segmented.

A general rule of thumb around which format to choose depends on three things:

- Will the table continue to grow at a fast rate?
- Am I working in a RAM/memory constrained environment?
- What level of performance do I want?

To learn more about these types and when to choose which <a href="https://code.kx.com/q/database/" target="_blank">see here</a>.

## 4 Scaling Dataset to 1 Billion Rows

In this section, we scale our dataset to 1 billion rows by duplicating an existing partition across multiple days. This approach ensures we have sufficient data for performance testing and analytics validation.

Before making copies, we check the disk space usage to ensure enough storage is available. The below system command displays the available and used disk space in megabytes (~9.6G), helping us monitor the impact of our operations.

In [10]:
system"df -mh ."

"Filesystem      Size  Used Avail Use% Mounted on"
"/dev/sdm        9.8G  183M  9.6G   2% /home/jovyan"


Next, we generate a list of new dates and copy the existing partition (2025.01.01) to these new dates. Here, we create 49 new partitions by copying the original partition for each additional day. This may take ~40 seconds to complete.

In [12]:
days:day +1 +til 49;                                         // Generate 49 additional days  
cmds: "cp -r ../data/2025.01.01 ../data/",/:string[days];    // Create shell commands to execute
\t system each cmds                                          // Execute shell commands to copy partitions  

55266


Once the partitions are created, we verify how much disk space was consumed and check the new partitions exist.

In [14]:
system"df -mh ."
system"ls -la ../data"

"Filesystem      Size  Used Avail Use% Mounted on"
"/dev/sdm        9.8G  8.9G  893M  92% /home/jovyan"
"total 212"
"drwxrwsr-x 52 jovyan users 4096 Mar  6 13:55 ."
"drwxrwsr-x  9 root   users 4096 Mar  6 10:20 .."
"drwxr-sr-x  3 jovyan users 4096 Mar  6 13:46 2025.01.01"
"drwxr-sr-x  3 jovyan users 4096 Mar  6 13:54 2025.01.02"
"drwxr-sr-x  3 jovyan users 4096 Mar  6 13:54 2025.01.03"
"drwxr-sr-x  3 jovyan users 4096 Mar  6 13:54 2025.01.04"
"drwxr-sr-x  3 jovyan users 4096 Mar  6 13:54 2025.01.05"
"drwxr-sr-x  3 jovyan users 4096 Mar  6 13:54 2025.01.06"
"drwxr-sr-x  3 jovyan users 4096 Mar  6 13:54 2025.01.07"
"drwxr-sr-x  3 jovyan users 4096 Mar  6 13:54 2025.01.08"
"drwxr-sr-x  3 jovyan users 4096 Mar  6 13:54 2025.01.09"
"drwxr-sr-x  3 jovyan users 4096 Mar  6 13:54 2025.01.10"
"drwxr-sr-x  3 jovyan users 4096 Mar  6 13:54 2025.01.11"
"drwxr-sr-x  3 jovyan users 4096 Mar  6 13:54 2025.01.12"
"drwxr-sr-x  3 jovyan users 4096 Mar  6 13:54 2025.01.13"
"drwxr-sr-x  3 jovyan users 409

Finially since kdb+ manages partitioned data at the filesystem level, we must reload the database to reflect the newly added partitions.

In [15]:
delete trade from `.                               // Delete in memory table
system"l ",dbDir                                   // Load the partitioned database
count trade                                        // 1B Rows 
select count i by date from trade                  // Select number of records per date within the trade table

.
1000000000
date      | x       
----------| --------
2025.01.01| 20000000
2025.01.02| 20000000
2025.01.03| 20000000
2025.01.04| 20000000
2025.01.05| 20000000
2025.01.06| 20000000
2025.01.07| 20000000
2025.01.08| 20000000
2025.01.09| 20000000
2025.01.10| 20000000
2025.01.11| 20000000
2025.01.12| 20000000
2025.01.13| 20000000
2025.01.14| 20000000
2025.01.15| 20000000
2025.01.16| 20000000
2025.01.17| 20000000
2025.01.18| 20000000
2025.01.19| 20000000
2025.01.20| 20000000
..


## 5. Time Series Analytics

Now that we have some data, let's dive into some basic time-series analytics.

### Total Trade Volume Every Hour

Let's find a symbol to analyse from the randomly generated list we created earlier and then run our query.

In [21]:
symbol:first syms
select sum size 
    by date,
       60 xbar time.minute 
    from trade 
    where sym=symbol

date       minute| size   
-----------------| -------
2025.01.01 00:00 | 4038856
2025.01.01 01:00 | 4214955
2025.01.01 02:00 | 4099337
2025.01.01 03:00 | 4113799
2025.01.01 04:00 | 4133509
2025.01.01 05:00 | 4071891
2025.01.01 06:00 | 4175473
2025.01.01 07:00 | 4130991
2025.01.01 08:00 | 4172108
2025.01.01 09:00 | 4106918
2025.01.01 10:00 | 4086639
2025.01.01 11:00 | 4153915
2025.01.01 12:00 | 4248459
2025.01.01 13:00 | 4113861
2025.01.01 14:00 | 4156215
2025.01.01 15:00 | 4121299
2025.01.01 16:00 | 4154079
2025.01.01 17:00 | 4118518
2025.01.01 18:00 | 4213332
2025.01.01 19:00 | 4199948
..


#### qSQL & Temporal Arithmetic
Here we are using <a href="https://code.kx.com/q/basics/qsql/" target="_blank">qSQL</a>, the inbuilt table query language in kdb+. If you have used SQL, you will find the syntax of qSQL queries very similar.
- Just as in SQL, table results called using `select` and `from` and can be filtered by expressions following a `where`
- Multiple filter criteria, separated by ,, are evaluated starting from the left
- To group similar values together we can use the `by` clause. This is particularly useful in combination with used with an aggregation like `sum`,`max`,`min` etc.

kdb+/q supports several temporal types and arithmetic between them. See here for a summary of <a href="https://code.kx.com/q/ref/#datatypes" target="_blank">datatypes</a>.

In this example:
- The `time` column in the data has a type of timestamp, which includes both date and time values.
- We convert the `time` values to their minute values (including hours and minutes)
- We then aggregate further on time by using <a href="https://code.kx.com/q/ref/xbar/" target="_blank">xbar</a> to bucket the minutes into hours (60-unit buckets)

### Weighted Average Price and Last Trade Price Every 15 Minutes 

In [23]:
select lastPx:last price, 
       vwapPx:size wavg price
 by date, 15 xbar time.minute 
 from trade 
 where sym=symbol

date       minute| lastPx   vwapPx  
-----------------| -----------------
2025.01.01 00:00 | 9.279603 49.68616
2025.01.01 00:15 | 93.50103 49.17632
2025.01.01 00:30 | 13.1666  50.20971
2025.01.01 00:45 | 37.99301 49.52158
2025.01.01 01:00 | 4.45246  49.35297
2025.01.01 01:15 | 38.27883 50.20757
2025.01.01 01:30 | 36.24516 50.12155
2025.01.01 01:45 | 21.79155 50.34161
2025.01.01 02:00 | 70.82111 49.8376 
2025.01.01 02:15 | 77.94584 49.15689
2025.01.01 02:30 | 45.79998 50.07452
2025.01.01 02:45 | 89.54561 49.95901
2025.01.01 03:00 | 4.223374 50.35866
2025.01.01 03:15 | 82.03917 49.60781
2025.01.01 03:30 | 18.97577 48.99254
2025.01.01 03:45 | 28.4487  51.41991
2025.01.01 04:00 | 11.68202 50.00599
2025.01.01 04:15 | 19.85271 49.92692
2025.01.01 04:30 | 9.235386 50.66791
2025.01.01 04:45 | 48.49472 49.65337
..


This is similar to the previous analytic, but this time we make use of the built in `wavg` function to find out the weighted average over time intervals. 

In finance, volume-weighted averages give a more accurate reflection of a stock’s price movement by incorporating trading volume at different price levels. This can be especially useful in understanding whether a price move is supported by strong market participation or is just a result of a few trades.

Let's time this anayltic with `\t` to see how long it takes in milliseconds to crunch through 1 Billion records.

In [24]:
\t select lastPx:last price, 
       vwapPx:size wavg price
   by date, 15 xbar time.minute 
   from trade 
   where sym=symbol

451


The query processed 1 Billion records in sub second time, efficiently aggregating last price (`lastPx`) and volume-weighted-average price (`vwapPx`) for MSFT trades. The use of `by date, 15 xbar time.minute` optimized the grouping, making the computation fast. This demonstrates the power of kdb+/q for high-speed time-series analytics.

 ### SQL Comparison

A SQL version of this query above would look something like:

```

SELECT 
    (array_agg(price ORDER BY time DESC))[1] AS lastPx,
    SUM(price * size) / NULLIF(SUM(size), 0) AS vwapPx,
    DATE_TRUNC('day', time),                                            
    TRUNC(time, 'MI') + (FLOOR(TO_NUMBER(TO_CHAR(time, 'MI')) / 15) * INTERVAL '15' MINUTE) 
FROM 
    trade
WHERE 
    sym = 'MSFT'
GROUP BY 
    DATE_TRUNC('day', time), 
    TRUNC(time, 'MI') + (FLOOR(TO_NUMBER(TO_CHAR(time, 'MI')) / 15) * INTERVAL '15' MINUTE)
ORDER BY 
    DATE_TRUNC('day', time), 
    TRUNC(time, 'MI') + (FLOOR(TO_NUMBER(TO_CHAR(time, 'MI')) / 15) * INTERVAL '15' MINUTE);

```

SQL is more complex due to several factors:
- **Time-series Calculations**: The SQL version involves the creation of custom logic for common time-series calculations such as volume-weighted-averages. In the q-sql version, these functionalities are implicit, and the syntax is more concise when working with vectors. The SQL equivalent requires custom definitions and is often more verbose leaving room for error.
- **Grouping and Aggregation**: In the q-sql version, grouping by date and a 15 minute window is done with a single, simple syntax, which is an efficient and intuitive way to express time bucketing. In SQL, similar behavior requires explicitly defining how time intervals are handled and aggregating the results using GROUP BY with custom time expressions which are often repeated throughout the query.
- **Temporal Formatting**: SQL queries often require repetitive conversion for handling timestamp formats, which is more cumbersome compared to q-sql, where time-based operations like xbar (interval-based bucketing) can be done directly in a streamlined manner. Temporal primitives also make it extremely easy to convert a nanosecond timestamp to it's equivalent minute using dot notation e.g. time.minute
- **Data Transformation**: The q language is optimized for high-performance, in-memory, columnar data transformations, which allows for more compact expressions on vectors of data. SQL, on the other hand, is typically too general purpose for even simple transformations on time-series data. This is down to how kdb+/q is designed, where operations execute on ordered lists, whereas SQL (based on set theory) treats data as records instead of columns e.g. selecting the (last) value in a series, or understanding prior states (deltas) for series movements would require re-ordering the column data
- **Performance Considerations**: q-sql is designed for high-performance analytics on large datasets, and many operations that would require complex SQL expressions can be done efficiently with q-sql syntax. In SQL, complex operations requires workarounds such as additional processing with temporary tables, sub-expressions, re-indexing, changing data models, or heavily leveraging partitions and window functions.

Thus, while the core logic of the query is similar in both languages, the SQL version requires much more overhead in terms of complexity and verbosity. This inefficiency will also become more pronounced with large datasets, leading to challenges with query performance.

While these are just basic analytics, they highlight kdb+/q’s ability to storage and analyse large-scale time-series datasets quickly.

## 6. Asof Join – Matching Trades with Quotes

One of the most powerful features in kdb+/q is the asof join (`aj`), which is designed to match records from two tables based on the most recent timestamp. Unlike a standard SQL join, where records must match exactly on a key, an asof join finds the most recent match.

Why Use Asof Joins?
In time-series data, we often deal with information arriving at different intervals. For example:
- Trade and Quote Data: A trade occurs at a given time, and we want to match it with the latest available quote.
- Sensor Data: A sensor records temperature every second, while another logs environmental data every 10 seconds—matching the closest reading is crucial.

> **📌** kdb+/q optimizes asof joins to handle large datasets efficiently, making it a key tool in real-time analytics and historical data analysis.

#### Generate synthetic quote data for one day

In [26]:
n:2000000
quote:([] 
    time:asc (`timestamp$day) + n?86400000000000;  // Random timestamps
    sym:n?syms;                                    // Random stock tickers
    bid:n?100f;                                    // Random bid prices
    ask:n?100f                                     // Random ask prices
 )

As we're keeping this table in memory we need to perform one extra step before joining, we apply the parted (p#) attribute to the sym column of the quote table. Our trade table on disk already has the parted attribute on the sym column, we see this in the column `a` when we run `meta trade`.

In [27]:
meta trade

c    | t f a
-----| -----
date | d    
sym  | s   p
time | p    
price| f    
size | j    


This is crucial for optimizing asof joins, as it ensures faster lookups when performing symbol-based joins. Before applying parted to quote, we first sort the table by sym using [`xasc`](#https://code.kx.com/q/ref/asc/), as the parted attribute requires the column to be sorted for it to work efficiently.

In [28]:
quote:`sym xasc quote                  / Sorting sym in ascending order
quote:update `p#sym from quote         / Apply the parted attruibute on sym

In the above:
- `xasc` Sorts the quote table by sym in ascending order
- `#`  Applies the parted attribute to sym, optimizing symbol-based lookups.

#### Peform Asof Join

We now match each trade with the most recent available quote for todays date using [`aj`](#https://code.kx.com/q/ref/aj/).


In [29]:
tradequote:aj[`sym`time; select from trade where date=day; quote]
tradequote

date       sym time                          price    size bid      ask     
----------------------------------------------------------------------------
2025.01.01 abj 2025.01.01D00:00:00.000000000 49.9322  112                   
2025.01.01 abj 2025.01.01D00:00:01.000000000 77.34705 780                   
2025.01.01 abj 2025.01.01D00:00:01.000000000 9.322147 266                   
2025.01.01 abj 2025.01.01D00:00:01.000000000 3.533422 634                   
2025.01.01 abj 2025.01.01D00:00:01.000000000 52.95851 729                   
2025.01.01 abj 2025.01.01D00:00:01.000000000 15.51982 86                    
2025.01.01 abj 2025.01.01D00:00:01.000000000 56.5147  393                   
2025.01.01 abj 2025.01.01D00:00:02.000000000 37.03945 164                   
2025.01.01 abj 2025.01.01D00:00:03.000000000 96.78387 765                   
2025.01.01 abj 2025.01.01D00:00:03.000000000 86.67517 899                   
2025.01.01 abj 2025.01.01D00:00:03.000000000 56.11418 396                   

In the above:
- `aj` performs an asof join on the `sym` and `time` columns
- Each trade record gets matched with the latest available quote at or before the trade’s timestamp.
- We can see this means the first few `bid` and `ask` values are empty because there was no quote data prior to those trades.

This approach ensures that for every trade, we have the best available quote information, allowing traders to analyze trade execution relative to the prevailing bid/ask spread at the time.

## Next Steps

Try [Example2](Example2.html) on Real-Time Ingestion & Streaming Analytics.
